<a href="https://colab.research.google.com/github/VirGonzalez/IA_/blob/main/Ejemplo_Py_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U py_plan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 14.3 MB/s eta 0:00:00


In [17]:
from operator import ne
from py_search.uninformed import depth_first_search
from py_search.uninformed import breadth_first_search
from py_search.uninformed import iterative_deepening_search
from py_plan.total_order import StateSpacePlanningProblem
from py_plan.base import Operator

# Definición de las acciones como instancias de Operator
check = Operator('check',
                 [('tiene_teoria', 'Geografia'),
                  ('tiene_teoria', 'Literatura'),
                  ('tiene_teoria', 'Matematica')],
                 [('leer', 'Geografia'),
                  ('leer', 'Literatura'),
                  ('leer', 'Matematica')])

resuelve_ejercicios = Operator('resuelve_ejercicios',
                               [('lee', 'Matematica'),
                                ('lee', 'Geografia'),
                                ('lee', 'Literatura')],
                               [('completa_ejercicios', 'Matematica'),
                                ('completa_ejercicios', 'Literatura'),
                                ('completa_ejercicios', 'Geografia'),
                                ('termina_tarea',)])

# Indica que el código dentro del if es ejecutado cuando el programa es corrido directamente por el interprete
# No, cuando es importado como un módulo
if __name__ == "__main__":
    # Definición del ESTADO INICIAL (lista de tuplas)
    start = [('tiene_teoria', 'Matematica'),   # tiene la teoria de matematicas
             ('tiene_teoria', 'Literatura'),   # tiene la teoria de literatura
             ('tiene_teoria', 'Geografia')     # tiene la teoria de geografia
             ]

    # Definición del estado final
    goal = [('completa_ejercicios', 'Matematica'),
            ('completa_ejercicios', 'Literatura'),
            ('completa_ejercicios', 'Geografia'),
            ('termina_tarea',)]

    # Progresión, búsqueda hacia adelante
    def progression(x):
        return breadth_first_search(x, forward=True, backward=False)

    # Definicion del problema con Estado Inicial, Estado Final y Acciones
    p = StateSpacePlanningProblem(start, goal, [check, resuelve_ejercicios])

    # Ejecución del planner y visualización de resultados
    print(next(progression(p)).path())

    # Ejecución del planner y visualización de resultados con Búsqueda iterativa en profundidad
    print(next(iterative_deepening_search(p)).path())

TypeError: ignored